# Project Milestone 4: Preliminary Results

## Imports 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif, SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

## Feature extraction 

Load the data.

In [2]:
data = pd.read_csv('mbti_1.csv')
data = data.replace({c: i for i, c in enumerate(pd.unique(data['type']))})
data.head()

,type,posts
0,0,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,1,'I'm finding the lack of me in these posts ver...
2,2,'Good one _____ https://www.youtube.com/wat...
3,3,"'Dear INTP, I enjoyed our conversation the o..."
4,4,'You're fired.|||That's another silly misconce...


Create feature matrix of raw word frequencies.

In [3]:
vectorizer = CountVectorizer(input='content')
X = vectorizer.fit_transform(list(data['posts']))
y = data['type'].values

print('n_classes =', X.shape[0])
print('n_features =', X.shape[1])

n_classes = 8675
n_features = 145412


Split into testing and training.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Training samples: {}'.format(len(y_train)))
print('Testing samples: {}'.format(len(y_test)))

Training samples: 6940
Testing samples: 1735


### Feature selection
See https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection. For example, we could use `mutual_info_classif` with `SelectKBest` to keep the $K$ best features according to their mutual information score. Or could do something else for now like PCA.

Select a dimensionality reduction technique below. Run only that section of cells. Then split into testing/training sets before running classification.

#### Mutual Information

In [5]:
m_info_scores = mutual_info_classif(X_train, y_train)
m_info_scores

array([4.63581620e-03, 4.02814192e-03, 5.26090599e-04, ...,
       2.98372438e-16, 2.77634655e-04, 2.94722181e-04])

The mutual info scores are relatively very low. There are only 6 features with a score greater than 0.1, 381 features with a score greater than 0.01, and 19037 features with a score greater than 0.001. The last threshold represents about 13% of the features, so I think that's a good starting point.

In [6]:
 k_num = np.count_nonzero(m_info_scores > 0.001)
 print(k_num)
 X_new = SelectKBest(chi2, k=k_num).fit_transform(X, y)
 print(X_new.shape)

19037
(8675, 19037)


#### PCA / Truncated Singular Value Decomp
X matrix is sparse but dense so PCA and SparsePCA does not work, needed to use TruncatedSVD

Found that 500 features explains above 86.7% of the variance. We can easily automate this to find some preferred error rate. 

In [7]:
pca = TruncatedSVD(n_components=500, n_iter=10, random_state=42)
X_new = pca.fit_transform(X, y)
print(X_new.shape)
print(pca.explained_variance_ratio_.sum())

(8675, 500)
0.8670950171056562


Re-split into testing and training after reducing dimensionality.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)
print('Training samples: {}'.format(len(y_train)))
print('Testing samples: {}'.format(len(y_test)))

Training samples: 6940
Testing samples: 1735


## Classification

Random guessing.

In [9]:
y_pred = np.random.randint(0, 16, size=y_test.shape)
print('Accuracy = {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy = 0.06


Support vector machine (linear kernel).
I assume we'll get better results ith TF-IDF vectors but do we need that for this step?

In [17]:
# clf = svm.LinearSVC()
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)
y_pred_lin = clf.predict(X_test)

SVM polynomial kernel

In [26]:
clf = svm.SVC(kernel='poly', C=1, gamma='auto')
clf.fit(X_train, y_train)
y_pred_pol = clf.predict(X_test)

## Performance evalutation
I suppose we should use $k$-fold cross validation.

Accuracy and cross validation scores vary across runs.
Comparison of linear and polynmomial kernels may not be valid
for these singular run results.

In [18]:
print('Accuracy = {:.2f}'.format(accuracy_score(y_test, y_pred_lin)))

Accuracy = 0.55


In [ ]:
print('Accuracy = {:.2f}'.format(accuracy_score(y_test, y_pred_pol)))

$k$-fold cross validation SVM linear kernel

In [25]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.55 (+/- 0.02)


$k$-fold cross validation ploynomial kernel

In [24]:
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.47 (+/- 0.04)
